In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Reading 2019-20 fpl data
fpl=pd.read_excel('../input/fantasy-premier-league-201920player-data/FPL_dataset.xlsx')


#According to fpl rules the squad must contain 2gk,5def,5mid and 3for,hence we create a 15 man team with above requirments.

In [ ]:

#print gk with most points
#gk=fpl[fpl["Position"]=="GK"]
#gk[gk["Points"]==gk.Points.max()
fpl=fpl[["Player","Team","Cost","Points","Position"]]

In [ ]:
#Creating a team of most valuable players
fpl["Worth"]=(fpl["Points"]/fpl["Cost"])

gk=fpl[fpl["Position"]=="GK"]
gk=gk.sort_values(by="Worth",ascending=False)

defence=fpl[fpl["Position"]=="DEF"]
defence=defence.sort_values(by="Worth",ascending=False)

mid=fpl[fpl["Position"]=="MID"]
mid=mid.sort_values(by="Worth",ascending=False)


forward=fpl[fpl["Position"]=="FOR"]
forward=forward.sort_values(by="Worth",ascending=False)

In [ ]:
mvp=gk[0:2]
mvp=pd.concat(([mvp,defence[0:5]]))
mvp=pd.concat((([mvp,mid[0:5]])))
mvp=pd.concat((([mvp,forward[0:3]])))
mvp

In [ ]:
print (mvp["Points"].sum())
print (mvp["Cost"].sum())

In [ ]:
#Creating a 11 of the most points
gk11=fpl[fpl["Position"]=="GK"]
gk11=gk11.sort_values(by="Points",ascending=False)

defence11=fpl[fpl["Position"]=="DEF"]
defence11=defence11.sort_values(by="Points",ascending=False)

mid11=fpl[fpl["Position"]=="MID"]
mid11=mid11.sort_values(by="Points",ascending=False)


forward11=fpl[fpl["Position"]=="FOR"]
forward11=forward11.sort_values(by="Points",ascending=False)

In [ ]:
dream11=gk11[0:2]
dream11=pd.concat(([dream11,defence11[0:5]]))
dream11=pd.concat((([dream11,mid11[0:5]])))
dream11=pd.concat((([dream11,forward11[0:3]])))
dream11

In [ ]:
print (dream11["Points"].sum())
print (dream11["Cost"].sum())

In [ ]:
#huge difference between points of mvp and best11 hence we need to build a team within net cost 100.
#also the team cant have more than 3 players of same team
#we use pulp to linearly optimize
!pip install -Iv pulp==1.6.8 --quiet
!pip install example --use-feature=2020-resolver

In [ ]:
#drop rows with low points to simplify calc
fpl= fpl[fpl['Points'] >50]
fpl

In [ ]:
#Creating required lists
import pulp
pointslist=fpl["Points"]
costlist=fpl["Cost"]
teamlist=fpl["Team"]
playerlist=fpl["Player"]
positionlist=fpl["Position"]

In [ ]:
#https://medium.com/@joseph.m.oconnor.88/linearly-optimising-fantasy-premier-league-teams-3b76e9694877
#used above website for ref

def select_team(points, cost, positions, teams, total_budget=100):
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    #Pulp problem which we will build
    
    num_players = len(points)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    #Objective function
    model += sum((decisions[i]) * points[i]
                 for i in range(num_players)), "Objective"
    
    model += sum(decisions[i] * cost[i] for i in range(num_players)) <= 100  # total cost
    model += sum(decisions) == 15  # total team size
                
    
    #2 goal keepers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == "GK") == 2
    
    #5 defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == "DEF") == 5
    
    #5 mids
    model += sum(decisions[i] for i in range(num_players) if positions[i] == "MID") == 5
    
    #3 forwards
    model += sum(decisions[i] for i in range(num_players) if positions[i] == "FOR") == 3
    
    #Club Constraints
    for club_id in np.unique(teams):
        model += sum(decisions[i]  for i in range(num_players) if teams[i] == club_id) <= 3 
    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return decisions
    

In [ ]:
decisions= select_team(pointslist.values, costlist.values, positionlist.values, teamlist.values)
indlist=[]

for i in range(0,len(decisions)):
    if decisions[i].value() != 0:
        indlist.append(i)
print (indlist)

index=[]
for i in range(0,199):
    index.append(i)
fpl.reindex(index)


fpl_best_team = pd.DataFrame() #df for the best team

for i in range(0,len(indlist)):
    n=([indlist[i]])
    fpl_best_team = fpl_best_team.append(fpl.iloc[n])
fpl_best_team

In [ ]:
print (fpl_best_team["Points"].sum())
print (fpl_best_team["Cost"].sum())